# Profiling and optimisation

### [Dr Marijan Beg](https://www.imperial.ac.uk/people/m.beg) :)

- BSc/MSc in Nanoelectronics / Nanotechnology / Nanomagnetism
- PhD in Computational Physics

### Research Interests:
- Magnetism and magnetic materials
- Computational magnetism - building simulation software used in academia and industry
- Topological quasi-particles and singularities in magnetisation fields (skyrmions, Bloch Points, hopfions...)
- Spintronics
    - Designing new data storage and information processing devices.
    
(Please feel free to approach me if you are interested in doing an IRP on any of these topics.)

## How to find me?

* [**Marijan Beg**](https://www.imperial.ac.uk/people/m.beg)
* email: m.beg@imperial.ac.uk
* GitHub: `@marijanbeg`
* Office: 4.95, Royal School of Mines (RSM, 4th floor)

## Learning objectives

By the end of this lecture you should be able to:

* Identify bottlenecks in your code by **profiling**
* Apply different **optimisation** techniques to speed up your code.
* Use general optimisation guidelines.

## Profiling: Measuring CPU time

### How long does it run?

- Let us compute the sum of the first billion natural numbers.

In [1]:
n = 1_000_000_000
s = sum(range(1, n+1))

assert s == n*(n+1)/2

### [`timeit`](https://docs.python.org/3/library/timeit.html) module

* Part of the Python's standard library (available wherever Python is installed)
* Provides a simple and quick way to time small bits of code
* Command-Line, Python, and "magic" IPython interface

### Command-Line Interface (CLI)

* We run it in Terminal/Command Prompt as:
```
$ python3 -m timeit "Python command"
```

In [2]:
!python3 -m timeit "sum(range(100))"

200000 loops, best of 5: 1.26 usec per loop


### Python interface

* We import `timeit` module, and use its `timeit` function (`timeit.timeit`).
* We pass a Python command as a string or as a callable.

In [3]:
import timeit

timeit.timeit('2 + 3')

0.02060143199923914

**HUH!?** Is it possible that `2 + 3` takes that long? Let us have a look at the documentation (please note the `number` parameter).

In [4]:
help(timeit.timeit)

Help on function timeit in module timeit:

timeit(stmt='pass', setup='pass', timer=<built-in function perf_counter>, number=1000000, globals=None)
    Convenience function to create Timer object and call timeit method.



`number` parameter specifies how many times the command is being executed.

In [5]:
timeit.timeit('2 + 3', number=1)

7.180060492828488e-07

#### Passing a callable to `timeit.timeit()`

In [6]:
def the_sum_of_2_and_3():
    return 2 + 3

timeit.timeit(the_sum_of_2_and_3, number=1)

1.2359960237517953e-06

### `%timeit` magic

* In IPython (Jupyter), we can use `%timeit` magic command.
* The output we receive is much more informative and "user-friendly".

In [7]:
%timeit 1 + 2

7.12 ns ± 0.158 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


* We should pay attention to standard deviation - if it it too large, we might want to repeat the measurement.
* The command `1 + 2` was executed in `r` runs, each consisting of `n` loops (`r * n` in total) to get better statistics.
* We can specify the number of runs (`-r`) and the number of iterations (`-n`).

In [8]:
%timeit -r 10 -n 100 1 + 2

8.82 ns ± 0.997 ns per loop (mean ± std. dev. of 10 runs, 100 loops each)


## `%timeit` magic on the entire cell

* How can we measure CPU time for multiple commands?

```
%%timeit setup

command1
command2
```

In [9]:
%%timeit import numpy as np

a = np.arange(100)
a**2

1.14 µs ± 3.11 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### `%timeit` output object

* Instead of printing the result, we can ask for the output object (`-o`).

In [10]:
import numpy as np
a = np.arange(100)

%timeit -o a**2

589 ns ± 2.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


<TimeitResult : 589 ns ± 2.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)>

Let us bind it to a variable:

In [11]:
timeit_object = %timeit -o a**2

575 ns ± 5.07 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


#### Measured times of individual runs

In [12]:
timeit_object.timings  # timings of individual runs

[5.742300620040623e-07,
 5.850273949909024e-07,
 5.705289969919249e-07,
 5.784574020071887e-07,
 5.689238640043186e-07,
 5.723167079995619e-07,
 5.730910080019385e-07]

#### A bit more statistics

In [13]:
timeit_object.average  # average time

5.74653633714271e-07

In [14]:
timeit_object.best  # the best (fastest) time

5.689238640043186e-07

In [15]:
timeit_object.worst  # the worst time

5.850273949909024e-07

In [16]:
timeit_object.stdev  # standard deviation

5.071050879787523e-09

## Exercise 1: Factorial - loop vs recursion

Write a function `factorial(n)` to calculate the factorial of a given number `n`. Write two versions of this function: one using recursion, and one using `for` loop. Use `%timeit` to decide which one performs better and by how much.

## `cProfile`

* So far, we measured total CPU time.
* How do we find a bottleneck? Where does our code spend most time?
* `cProfile` gives us statistics on how many times each function was called and how much time was spent in each.

### Example: Estimate $\pi$ using Monte Carlo simulation

Let us have a look at the example where we are trying to estimate $\pi$ value by running a Monte Carlo simulation. For more details, please refer to: https://www.101computing.net/estimating-pi-using-the-monte-carlo-method/

In [21]:
import math
import random


def random_list(n):
    """Create a list with n elements. Each
    element has value between -1 and 1.
    
    Parameters
    ----------
    n : int
        The number of elements.
        
    Return
    ------
    list(float)
    
    """
    return [2*random.random() - 1 for _ in range(n)]

In [22]:
def distance(x, y):
    """Compute the distance from the origin (0, 0)
    of a point with coordinates x and y.
    
    Parameters
    ----------
    n : int
        The number of elements.
        
    Return
    ------
    float
        Distance from origin.

    """
    return math.sqrt(x**2 + y**2)

In [23]:
def estimate_pi(n):
    """Estimate the value of pi using Monte Carlo simulation.
    
    Parameters
    ----------
    n : int
        The number of random points generated.
        
    Returns
    -------
    float
        An estimate of pi.

    """
    x_list = random_list(n)
    y_list = random_list(n)
    
    in_circle = 0
    for x, y in zip(x_list, y_list):
        if distance(x, y) <= 1:
            in_circle += 1
            
    return 4*in_circle / n

### Total execution time

In [24]:
%timeit estimate_pi(n=1_000_000)

829 ms ± 8.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


By measuring the total execution time, we do not know where exactly the bottleneck of our code could be.

## `cProfile`

In [25]:
import cProfile

profiler = cProfile.Profile()
profiler.runcall(estimate_pi, n=1_000_000)
profiler.print_stats()

         4000006 function calls in 1.508 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.296    0.296    1.508    1.508 1371357974.py:1(estimate_pi)
  1000000    0.422    0.000    0.505    0.000 1732563340.py:1(distance)
        2    0.553    0.276    0.707    0.354 2212784721.py:19(<listcomp>)
        2    0.000    0.000    0.707    0.354 2212784721.py:5(random_list)
  1000000    0.083    0.000    0.083    0.000 {built-in method math.sqrt}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
  2000000    0.155    0.000    0.155    0.000 {method 'random' of '_random.Random' objects}




### Saving statistics

In [26]:
filename = 'stats.prof'
profiler.dump_stats(filename)

### Visualising `cProfile` results

* We can use `snakeviz` to visualise and better understand the results.

```
    $ snakeviz "stats.prof" &
```

### `%prun` magic

- Similar to `timeit`, we can run profiler using a magic command `%prun`.

In [27]:
%prun estimate_pi(n=1_000_000)

         4000008 function calls in 1.498 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.528    0.264    0.676    0.338 2212784721.py:19(<listcomp>)
  1000000    0.409    0.000    0.492    0.000 1732563340.py:1(distance)
        1    0.297    0.297    1.464    1.464 1371357974.py:1(estimate_pi)
  2000000    0.148    0.000    0.148    0.000 {method 'random' of '_random.Random' objects}
  1000000    0.083    0.000    0.083    0.000 {built-in method math.sqrt}
        1    0.034    0.034    1.498    1.498 <string>:1(<module>)
        1    0.000    0.000    1.498    1.498 {built-in method builtins.exec}
        2    0.000    0.000    0.676    0.338 2212784721.py:5(random_list)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

## Exercise 2: `cProfile`

Profile separately using `cProfile` functions `slow_sum_list` and `fast_sum_list` from the following cell. Compare and intepret the results.

In [28]:
import time


def fast_sum(x, y):
    time.sleep(0.01)  # wait 0.01 seconds
    return x + y

def slow_sum(x, y):
    time.sleep(0.1)  # wait 0.1 seconds
    return x + y

def fast_sum_list(x_list, y_list):
    res = []
    for x, y in zip(x_list, y_list):
        res.append(fast_sum(x, y))
        
    return res

def slow_sum_list(x_list, y_list):
    res = []
    for x, y in zip(x_list, y_list):
        res.append(slow_sum(x, y))
        
    return res

x_list = list(range(100))
y_list = list(range(100))

### `line_profiler`

- `cProfile` told us how many times each function was called and how much time was spent in it.
- To investigate even further, we can measure how much time was spent for each line of code.
- For that, we use `line_profiler`
    - You can install it by running `pip install line_profiler`
- In `.py` file, we decorate functions we want to inspect with `@profile`.

In [31]:
# %load line_profiler_demo.py
# import math
# import random


# @profile
# def random_list(n):
#     return [2*random.random() - 1 for _ in range(n)]


# @profile
# def distance(x, y):
#     return math.sqrt(x**2 + y**2)


# @profile
# def estimate_pi(n):
#     x_list = random_list(n)
#     y_list = random_list(n)
    
#     in_circle = 0
#     for x, y in zip(x_list, y_list):
#         if distance(x, y) <= 1:
#             in_circle += 1
            
#     return 4*in_circle / n

# if __name__ == '__main__':
#     estimate_pi(1_000_000)

In [32]:
!kernprof -l -v line_profiler_demo.py

Wrote profile results to line_profiler_demo.py.lprof
Timer unit: 1e-06 s

Total time: 0.747942 s
File: line_profiler_demo.py
Function: random_list at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           @profile
     6                                           def random_list(n):
     7         2     747942.0 373971.0    100.0      return [2*random.random() - 1 for _ in range(n)]

Total time: 0.667735 s
File: line_profiler_demo.py
Function: distance at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           @profile
    11                                           def distance(x, y):
    12   1000000     667735.0      0.7    100.0      return math.sqrt(x**2 + y**2)

Total time: 4.56641 s
File: line_profiler_demo.py
Function: estimate_pi at line 15

Line #      Hits         Time  Per Hit   % Time  Line Contents
    15                                        

## Memory profiler

- Besides the CPU time, we sometimes need to measure the memory usage.
- For that, we can use `memory_profiler`
    - You can install it by running `pip install memory_profiler`
- After loading an extension, we can run `memory_profiler` as:

In [33]:
%load_ext memory_profiler

### `%memit`

- Memory equivalent of `%timeit`.

In [34]:
%memit estimate_pi(n=1_000_000)

peak memory: 158.41 MiB, increment: 76.41 MiB


### `%mprun`

- Memory equivalent of `%lprun`.
- Unfortunately, it works only on code in separate modules, not the code in notebook.

In [35]:
%%file mprun_demo.py
def sum_of_lists(n):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(n)]
        total += sum(L)
        del L # remove reference to L
    return total

# Example taken from: https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

Overwriting mprun_demo.py


In [36]:
# from mprun_demo import sum_of_lists
# %mprun -f sum_of_lists sum_of_lists(1000000)

## Optimisation - making the code run faster

*“Premature optimisation is the root of all evil”* - [Sir Tony Hoare](https://en.wikipedia.org/wiki/Tony_Hoare) (or [Donald Knuth](https://en.wikipedia.org/wiki/Donald_Knuth))

## When we (should) optimise

1. **Make it work**: write the code in a simple, readable, and maintainable way.
2. **Make it reliable**: write tests and make sure that if we break the code, the tests will let us know
3. **Measure, measure, measure**: Measure the performance, do not guess!
4. **Convince ourselves that our code is too slow**: Does it hurt if it runs too slow?
5. **Consider the costs of optimisation**: Hardware is cheaper than the developer's time.
6. **Optimise**

In [37]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


### General gudelines

1. Convince yourself that your program is too slow.
    - Does it hurt if the program runs slowly?
    - Is it slow for a realistic use case?
    - Are there any external causes (network, IO, ...)
    - Find bottlenecks by profiling - measure, don't guess!
    - *One accurate measurement is worth a thousand expert opinions.* - **Grace Hopper**
2. Optimisation costs.
    - Optimisation is time consuming.
    - How much readibility and maintainability will suffer?
    - Is it cheaper to use better hardware (hardware is cheaper than the developer time).
3. Do not optimize while you develop
    - Correct first, fast later.
4. Always check the result of optimisation (unit tests, coverage)

### Algorithmic optimisation

- Generally, it is the first thing to look for.
- Are there any ways to compute less?
- Can we use an algorithm with different $O(N)$?

### Example

How could we rewrite the following code to run faster?

**Hint:** Have a look at Python `set` datatype and its operators (https://realpython.com/python-sets/).

In [38]:
import numpy as np

x = np.random.randint(0, 1000, 10000)
y = np.random.randint(0, 1000, 10000)

def find_common_elements(x, y):
    """Find common elements in arrays x and y."""
    res = []
    for i in x:
        for j in y:
            if i==j and i not in res:
                res.append(i)
    
    return res

In [39]:
%timeit -n 1 -r 1 find_common_elements(x, y)

8.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


If we use sets:

In [40]:
def using_sets(x, y):
    return list(set(x) & set(y))

In [41]:
%timeit using_sets(x, y)

1.77 ms ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Let NumPy/pandas do the work

"Vectorization is the process of converting an algorithm from operating on a single value at a time to operating on a set of values at one time. Modern CPUs provide direct support for vector operations where a single instruction is applied to multiple data (SIMD)." [Intel Vectorization](https://www.intel.com/content/www/us/en/developer/articles/technical/vectorization-a-key-tool-to-improve-performance-on-modern-cpus.html#:~:text=Vectorization%20is%20the%20process%20of,to%20multiple%20data%20(SIMD).)

### Example 1: Computing the mean

In [42]:
import pandas as pd

A = np.random.rand(100, 100)
data = pd.DataFrame(A)

def mean_loops(A):
    s = 0
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            s += A[i, j]
            
    return s / A.size

def mean_numpy(A):
    return A.mean()  # or np.mean(A)

In [43]:
%timeit mean_loops(A)

1.93 ms ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%timeit mean_numpy(A)

8.19 µs ± 41.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Example 2: Matrix-matrix multiplication

In [45]:
A = np.random.rand(100, 100)
B = np.random.rand(100, 100)

def matmult_loops(A, B):
    res = np.zeros([A.shape[0], B.shape[1]])
    for i in range(res.shape[0]):
        for j in range(res.shape[1]):
            for k in range(res.shape[1]):
                res[i, j] += A[i, k] * B[k, j]

    return res

def matmult_numpy(A, B):
    return A @ B  # np.matmult(A, B)

assert np.allclose(matmult_loops(A, B), A @ B)

In [46]:
%timeit matmult_loops(A, B)

617 ms ± 6.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%timeit matmult_numpy(A, B)

85.4 µs ± 857 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Memoisation (caching) - do not compute the same thing twice.

### Example 3: Fibonacci

In [48]:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [49]:
%timeit fibonacci(35)

3.61 s ± 42.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can use `cache` to save the results of previous calls.

In [50]:
import functools

@functools.cache
def fibonacci(n):
    if n == 0:  # There is no 0'th number
        return 0
    elif n == 1:  # We define the first number as 1
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

In [51]:
%timeit fibonacci(35)

69 ns ± 0.931 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## Exercise 3: Caching

In one of the previous exercises, we wrote a function for computing factorial using recursion. Measure how much the performance improves if caching is used. Measure performance for `n=100`.

## Exercise 4: Code optimisation

In the following cell is the code for estimating  and we used it previously to demonstrate profiling.

Measure the CPU time of estimate_pi(1_000_000) and write it down.
By profiling and optimisation, how much were you able to reduce CPU time?

In [55]:
# %load estimate_pi.py
import math
import random


def random_list(n):
    """Create a list with n elements, with each
    element with value between -1 and 1.
    
    Parameters
    ----------
    n : int
        The number of elements.
        
    Return
    ------
    list(float)
    
    """
    return [2*random.random() - 1 for _ in range(n)]


def distance(x, y):
    """Compute the distance from the origin of a point with coordinates x and y.
    
    Parameters
    ----------
    n : int
        The number of elements.
        
    Return
    ------
    float
        Distance from origin.

    """
    return math.sqrt(x**2 + y**2)


def estimate_pi(n):
    """Estimate the value of pi using Monte Carlo simulation.
    
    Parameters
    ----------
    n : int
        The number of random points generated.
        
    Returns
    -------
    float
        An estimate of pi.

    """
    x_list = random_list(n)
    y_list = random_list(n)
    
    in_circle = 0
    for x, y in zip(x_list, y_list):
        if distance(x, y) <= 1:
            in_circle += 1
            
    return 4*in_circle / n

In [56]:
%timeit estimate_pi(1_000_000)

833 ms ± 3.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## My code is still slow. What now?

* Replace as much of your code with third-party libraries - they are likely better optimised than what you had time for. 
* Are you using all the cores in your system? Following step 1 above first will likely mean this is true. If you still see only one core being used, you need to parallelise your program. Tomorrow's lecture will introduce that. 
* You could transfer the hot spots, i.e. the few lines or functions in which most of the time is spent, to compiled code ([numba](https://numba.pydata.org) or [Cython](https://cython.org/)).